In [1]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import progressbar

In [2]:
class RBM(object):
    """Restricted Boltzmann Machine"""
    def __init__(self, n_visible=4096, n_hidden=400, W=None, hbias=None, vbias=None, n_epochs=100, eta=0.0001, batch_size=128, random_state=42):
        """
        W: shape = [n_hidden, n_visible]
        hbias: shape = [n_hidden, ]
        vbias: shape = [n_visible, ]
        """
        self.random = np.random.RandomState(random_state)
        
        self.n_visible = tf.constant(n_visible)
        self.n_hidden = tf.constant(n_hidden)
        self.n_epochs = tf.constant(n_epochs)
        self.eta = tf.constant(eta) # learning rate
        self.batch_size = tf.constant(batch_size)
        self.eval_ = {'cross_entropy':[], 'pseudo_likelihood':[], 'visible_energy':[]}

        if W is None:
            W = tf.Variable(tf.random.normal([n_hidden, n_visible],
                                             mean=0,stddev=tf.sqrt(2/float(n_hidden+n_visible))), dtype='float32')
        if hbias is None:
            hbias = tf.Variable(tf.zeros([n_hidden]), dtype='float32')
        if vbias is None:
            vbias = tf.Variable(tf.zeros([n_visible]), dtype='float32')

        self.W = W
        self.hbias = hbias
        self.vbias = vbias
        self.params = [self.W, self.hbias, self.vbias]

    def propup(self, vis):
        """return p(h|v)
        """
        wxv_c = (tf.matmul(vis, tf.transpose(self.W)) + self.hbias)*2 # [batch_size, n_hidden] 
        return self._sigmoid(wxv_c)

    def v_to_h(self, v0_sample):
        """v -> h, sample h given v from p(h|v)
        """
        h1_mean = self.propup(v0_sample)
        h1_sample = (self.random.binomial(n=1, p=h1_mean, size=h1_mean.shape)-0.5)*2
        return tf.Variable(h1_sample,dtype='float32')

    def propdown(self, hid):
        """return p(v|h)
        """
        hxw_b = (tf.matmul(hid, self.W) + self.vbias)*2
        return self._sigmoid(hxw_b)

    def h_to_v(self, h0_sample):
        """h -> v, sample v given h from p(v|h)
        """
        v1_mean = self.propdown(h0_sample)
        v1_sample = (self.random.binomial(n=1, p=v1_mean, size=v1_mean.shape)-0.5)*2
        return tf.Variable(v1_sample,dtype='float32')

    def gibbs_update(self, v_sample, k=1):
        """gibbs sampling
        CD-k to obtain batch_size number of visible states
        """
        for step in range(k):
            h_sample = self.v_to_h(v_sample)
            v_sample = self.h_to_v(h_sample)
        return v_sample

    def fit(self, X):
        """training of RBM
        """
        m_instances = X.shape[0]
        n_batches = m_instances // self.batch_size.numpy()
        bar = progressbar.ProgressBar(maxval=n_batches,widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
        
        for epoch in range(self.n_epochs):
            bar.start()
            for batch_index in range(n_batches):
                indices = np.random.randint(m_instances, size=self.batch_size) #stochastic gradient descent
                v_batch = tf.Variable(X[indices, :],dtype='float32')

                # positive phase 
                with tf.GradientTape(persistent = True) as tp:
                    loss = tf.reduce_mean(-tf.reshape(tf.matmul(v_batch, tf.reshape(self.vbias,[self.n_visible,1])), [v_batch.shape[0]]) 
                                           - tf.reduce_sum( tf.math.log(tf.math.exp(tf.clip_by_value(-(tf.matmul(v_batch,tf.transpose(self.W)) + self.hbias), -50., 50)) + 
                                                  tf.math.exp(tf.clip_by_value(tf.matmul(v_batch,tf.transpose(self.W)) + self.hbias, -50., 50))), axis = 1 ))
                dW = tp.gradient(loss, self.W)
                dhbias = tp.gradient(loss, self.hbias)
                dvbias = tp.gradient(loss, self.vbias)
                del tp

                # negative phase
                v_sample = self.gibbs_update(v_batch, k=5)
                with tf.GradientTape(persistent = True) as tp:
                    loss = tf.reduce_mean(-tf.reshape(tf.matmul(v_sample, tf.reshape(self.vbias,[self.n_visible,1])), [v_sample.shape[0]]) 
                                           - tf.reduce_sum( tf.math.log(tf.math.exp(tf.clip_by_value(-(tf.matmul(v_sample,tf.transpose(self.W)) + self.hbias), -50., 50)) + 
                                                  tf.math.exp(tf.clip_by_value(tf.matmul(v_sample,tf.transpose(self.W)) + self.hbias, -50., 50))), axis = 1 ))
                dW -= tp.gradient(loss, self.W)
                dhbias -= tp.gradient(loss, self.hbias)
                dvbias -= tp.gradient(loss, self.vbias)
                del tp
                
                self.W.assign_sub(self.eta * dW)
                self.hbias.assign_sub(self.eta * dhbias )     
                self.vbias.assign_sub(self.eta * dvbias)

                bar.update(batch_index+1)
            bar.finish()

            CE = self.cross_entropy(X)
            L = self.pseudo_likelihood(X)
            visible_energy = tf.reduce_mean(self.visible_energy(X)).numpy()
            self.eval_['cross_entropy'].append(CE)
            self.eval_['pseudo_likelihood'].append(L)
            self.eval_['visible_energy'].append(visible_energy)      
            print('epoch:', epoch, 'cross entropy:', CE, 'pseudo-likelihood:', L, 'visible energy:', visible_energy)
        return self

    def _sigmoid(self, z):
        """ logistic function
        """
        return 1. / (1. + tf.math.exp(-tf.clip_by_value(z, -250, 250)))

    def cross_entropy(self,v_input):
        """return mean of cross entropy
        """
        v_input = tf.Variable(v_input,dtype='float32')
        h = self.v_to_h(v_input)
        p = self.propdown(h)
        
        v_input = v_input/2+0.5
        J = v_input*tf.math.log(tf.clip_by_value(p,0.000001,1)) + (1-v_input)*tf.math.log(tf.clip_by_value(1-p,0.000001,1))
        return -tf.reduce_mean(J).numpy()
    
    def pseudo_likelihood(self,v_input):
        """return mean of pseudo-likelihood
        """
        v = []
        v_ = []
        for i in range(v_input.shape[0]):
            idx = np.random.randint(0,v_input.shape[1])
            v_copy = v_input[i].copy()
            v_copy[idx] *= (-1)
            v_.append(v_copy)

        e = self.visible_energy(tf.Variable(v_input,dtype='float32'))
        e_ = self.visible_energy(tf.Variable(v_,dtype='float32'))
        PL = -self.n_visible.numpy() * np.log(self._sigmoid(e_-e).numpy())
        return np.mean(PL)
        
    def visible_energy(self, v_sample):
        """return batch_size number of visible energy 
        """
        v_sample = tf.Variable(v_sample,dtype='float32')
        wxv_c =  tf.matmul(v_sample,tf.transpose(self.W)) + self.hbias
        vbias_term = tf.reshape(tf.matmul(v_sample, tf.reshape(self.vbias,[self.n_visible,1])), [v_sample.shape[0]])
        hidden_term = tf.reduce_sum( tf.math.log(tf.math.exp(tf.clip_by_value(-wxv_c, -50., 50)) + 
                                                 tf.math.exp(tf.clip_by_value(wxv_c, -50., 50))), axis = 1 )
        return -vbias_term-hidden_term


In [3]:
path = '/home/jg481/MCIsingData/'
X=np.load(path+'T225.npy')

In [5]:
rbm = RBM(n_visible=4096, n_hidden=400, n_epochs=5, eta=0.0001, batch_size=128, random_state=42)
rbm.fit(X)

[========================================================================] 100%


epoch: 0 cross entropy: 0.43575454 pseudo-likelihood: 1787.8601 visible energy: -2313.8206


epoch: 1 cross entropy: 0.4354165 pseudo-likelihood: 1784.2795 visible energy: -2318.1335


epoch: 2 cross entropy: 0.43485507 pseudo-likelihood: 1793.8171 visible energy: -2330.8418


epoch: 3 cross entropy: 0.43401247 pseudo-likelihood: 1784.7045 visible energy: -2330.8865


epoch: 4 cross entropy: 0.43264118 pseudo-likelihood: 1798.2748 visible energy: -2344.7876
